In [1]:
!rm -r data_zip
!rm -r data
!mkdir data_zip
!mkdir data
import urllib.request
url = 'http://theremin.music.uiowa.edu/sound%20files/MIS%20Pitches%20-%202014/Strings/'
instruments = ['Violin','Viola','Cello']
attacks = ['arco','pizz']
strings = {'Violin':['E','A','D','G'], 'Viola':['A','D','G','C'], 'Cello':['A','D','G','C']}

for instrument in instruments:
  print('Downloading ', instrument, ' files')
  for attack in attacks:
    for string in strings[instrument]:
      filename = instrument+'.'+attack+'.ff.sul'+string+'.stereo.zip'
      fullpath = url+instrument+'/'+filename
      response = urllib.request.urlretrieve(fullpath,'data_zip/'+filename)

!for file in data_zip/*.zip; do unzip "$file" -d data; done
!for file in data/*.aif; do ffmpeg -i "$file" "$file".wav; done
!rm data/*.aif 
!rm -r data/__MACOSX

import glob
import librosa    
x_arco = []
for filename in glob.glob('data/*arco*.wav'):
  data, fs = librosa.load(filename, sr=16000)  
  x_arco.append(data)

x_pizz = []
for filename in glob.glob('data/*pizz*.wav'):
  data, fs = librosa.load(filename, sr=16000)
  x_pizz.append(data)

import numpy as np
x = x_arco + x_pizz
max_len = 16000
for idata, data in enumerate(x):
  x[idata] = np.ndarray.tolist(data[:max_len]) + [0]*(max_len - len(data))

X = np.array(x)
y = np.array([0]*len(x_arco) + [1]*len(x_pizz))

print('The shape of X is: ', X.shape)
print('The shape of y is: ', y.shape)

Streaming output truncated to the last 5000 lines.
Guessed Channel Layout for Input Stream #0.0 : stereo
Input #0, aiff, from 'data/Violin.arco.ff.sulA.Eb5.stereo.aif':
  Metadata:
    comment         : 
  Duration: 00:00:02.51, start: 0.000000, bitrate: 2118 kb/s
    Stream #0:0: Audio: pcm_s24be, 44100 Hz, stereo, s32 (24 bit), 2116 kb/s
Stream mapping:
  Stream #0:0 -> #0:0 (pcm_s24be (native) -> pcm_s16le (native))
Press [q] to stop, [?] for help
Output #0, wav, to 'data/Violin.arco.ff.sulA.Eb5.stereo.aif.wav':
  Metadata:
    ICMT            : 
    ISFT            : Lavf57.83.100
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, stereo, s16, 1411 kb/s
    Metadata:
      encoder         : Lavc57.107.100 pcm_s16le
size=     433kB time=00:00:02.51 bitrate=1411.4kbits/s speed= 806x    
video:0kB audio:433kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.017592%
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  bui

In [2]:
# let's extract some features, because otherwise this does not make any sense!
import librosa

X_ft = np.array([[np.mean(librosa.feature.spectral_centroid(x)),
                  np.mean(librosa.feature.rms(x))
                  ] for x in X])
print("The shape of X_ft: ", X_ft.shape)

The shape of X_ft:  (577, 2)


In [3]:
# IOWA musical instruments dataset
print("the shape of X_ft is: ", X_ft.shape) # <- this is the data
print("the shape of y is: ", y.shape) # <- these are the labels
fs = 16000
# let's hear
import IPython.display as ipd
display(ipd.Audio(X[-1], rate=16000))

# how many datapoints do we have?
Ntotal = X.shape[0]
Ntest = int(Ntotal*0.1)
Ntrain = Ntotal - Ntest

np.random.seed(0)
# we can use np.random.choice to randomize the indices of all the data
shuff_idx = np.random.choice(Ntotal, Ntotal, replace=False)

# Let's separate the testing data 
X_ts = X_ft[shuff_idx[:Ntest]]
y_ts = y[shuff_idx[:Ntest]]

# now let's separate the training data
X_train = X_ft[shuff_idx[Ntest:]]
y_train = y[shuff_idx[Ntest:]]

print("the number of testing datapoints is: ", Ntest)
print("the number of training datapoints is: ", Ntrain)

# we still have to split this into folds
number_of_folds = 5
Nfold = int(Ntrain/number_of_folds)
X_folds = []
y_folds = []

for ifold in range(number_of_folds):
  X_folds.append(X_train[ifold*Nfold:ifold*Nfold+Nfold])
  y_folds.append(y_train[ifold*Nfold:ifold*Nfold+Nfold])

print("The shape of X_ts is: ", X_ts.shape)
print("The shape of y_ts is: ", y_ts.shape)
[print("The shape data in  fold No. ",i," is : ", x.shape) for i, x in enumerate(X_folds)]
[print("The shape labels in fold No. ",i," is: ", y.shape) for i, y in enumerate(y_folds)]

the shape of X_ft is:  (577, 2)
the shape of y is:  (577,)


the number of testing datapoints is:  57
the number of training datapoints is:  520
The shape of X_ts is:  (57, 2)
The shape of y_ts is:  (57,)
The shape data in  fold No.  0  is :  (104, 2)
The shape data in  fold No.  1  is :  (104, 2)
The shape data in  fold No.  2  is :  (104, 2)
The shape data in  fold No.  3  is :  (104, 2)
The shape data in  fold No.  4  is :  (104, 2)
The shape labels in fold No.  0  is:  (104,)
The shape labels in fold No.  1  is:  (104,)
The shape labels in fold No.  2  is:  (104,)
The shape labels in fold No.  3  is:  (104,)
The shape labels in fold No.  4  is:  (104,)


[None, None, None, None, None]

In [4]:
import numpy as np
# variables that we will need

D = X_folds[0].shape[1]
lr = 1.0
epochs = 10

# regression function
def sigmoid(theta):
  return np.exp(theta)/(1+np.exp(theta))

J_tr_folds = []
J_vl_folds = []

for ifold in range(number_of_folds):
  print("the current validation fold is: ", ifold)

  v = np.random.randn(D+1) # <- parameters that we want to train

  X_vl = X_folds[ifold]
  y_vl = y_folds[ifold]

  
  X_tr = np.vstack([X_folds[i] for i in range(number_of_folds) if i != ifold])
  y_tr = np.hstack([y_folds[i] for i in range(number_of_folds) if i != ifold])
  X_tr_mean = np.mean(X_tr, axis=0, keepdims=True)
  X_tr_std = np.std(X_tr, axis=0, keepdims=True)
  X_tr = X_tr - X_tr_mean
  X_tr /= X_tr_std
  X_tr = np.hstack([X_tr, np.ones((X_tr.shape[0],1))])

  X_vl = X_vl - X_tr_mean
  X_vl /= X_tr_std
  X_vl = np.hstack([X_vl, np.ones((X_vl.shape[0],1))])


  # getting the number of training and validation datapoints  
  N_tr = X_tr.shape[0]
  N_vl = X_vl.shape[0]

  for epoch in range(epochs):
    
    # let's find y_hat_tr
    y_hat_tr = sigmoid(np.dot(X_tr,v))
    # print(y_hat_tr)
    
    # let's find y_hat_vl
    y_hat_vl = sigmoid(np.dot(X_vl,v))

    # compute the cost for the training data
    J_tr = -(1/N_tr)*np.sum((y_tr*np.log(y_hat_tr) + (1-y_tr)*np.log(1-y_hat_tr)))
    J_vl = -(1/N_vl)*np.sum((y_vl*np.log(y_hat_vl) + (1-y_vl)*np.log(1-y_hat_vl)))    

    # now we need to update v with the gradients to optimize
    dJdyhat = y_hat_tr - y_tr
    dJdv = (1/N_tr)*np.dot(dJdyhat.T,X_tr)

    v -= lr*dJdv

  J_tr_folds.append(J_tr)
  J_vl_folds.append(J_vl)

print("The mean J_tr after ", epochs, " epochs was: ", np.mean(J_tr_folds))
print("The mean J_vl after ", epochs, " epochs was: ", np.mean(J_vl_folds))

the current validation fold is:  0
the current validation fold is:  1
the current validation fold is:  2
the current validation fold is:  3
the current validation fold is:  4
The mean J_tr after  10  epochs was:  0.18011703598302936
The mean J_vl after  10  epochs was:  0.17549389594078119
